In [ ]:
%cp '/content/drive/My Drive/data/train/labels_cocoformat.json' '/content/'
%mv '/content/labels_cocoformat.json' '/content/train_labels_cocoformat.json'
%cp '/content/drive/My Drive/data/val/labels_cocoformat.json' '/content/'
%mv '/content/labels_cocoformat.json' '/content/val_labels_cocoformat.json'

In [ ]:
%cp '/content/drive/My Drive/fisheye-day-30062020.zip' '/content/'

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/fisheye-day-30062020.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
!nvidia-smi

Fri Oct 23 15:53:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os

os.mkdir('/content/train_images')
#os.mkdir('/content/train')
os.mkdir('/content/val_images')
#os.mkdir('/content/val_labels')

In [ ]:
f = open('/content/training_list.txt','r')

l = f.readlines()
import shutil
from tqdm import tqdm
img_source = '/content/fisheye-day-30062020/images/train/'
label_source = '/content/fisheye-day-30062020/labels/train/'
img_dest = '/content/train_images/'
label_dest = '/content/train_labels/'

for i in tqdm(range(len(l))):
    l[i] = l[i].split('\n')[0]
    shutil.copyfile(img_source + l[i] , img_dest+ l[i])
    #label_name = l[i].split('.')[0] + '.txt'
    #shutil.copyfile(label_source + label_name, label_dest + label_name)
f.close()

100%|██████████| 14473/14473 [01:13<00:00, 196.19it/s]


In [ ]:
f = open('/content/validation_list.txt','r')

l = f.readlines()
import shutil
from tqdm import tqdm
img_source = '/content/fisheye-day-30062020/images/train/'
label_source = '/content/fisheye-day-30062020/labels/train/'
img_dest = '/content/val_images/'
label_dest = '/content/val_labels/'

for i in tqdm(range(len(l))):
    l[i] = l[i].split('\n')[0]
    shutil.copyfile(img_source + l[i] , img_dest+ l[i])
    #label_name = l[i].split('.')[0] + '.txt'
    #shutil.copyfile(label_source + label_name, label_dest + label_name)
f.close()

100%|██████████| 5348/5348 [00:50<00:00, 105.49it/s]


In [ ]:
%%capture
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:

!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab


1.5.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
%%capture
# install detectron2: (colab has CUDA 10.1 + torch 1.5)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.5")
!pip install detectron2==0.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances('vip_train', {}, '/content/train_labels_cocoformat.json', '/content/train_images')
register_coco_instances('vip_val', {}, '/content/val_labels_cocoformat.json', '/content/val_images')

In [ ]:
from detectron2.data import MetadataCatalog
MetadataCatalog.get('vip_train').thing_classes = ['vehicle']
MetadataCatalog.get('vip_val').thing_classes = ['vehicle']

In [ ]:
from detectron2.engine import DefaultTrainer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/retinanet_R_101_FPN_3x.yaml'))
cfg.DATASETS.TRAIN = ('vip_train',)
cfg.DATASETS.TEST = ('vip_val',)
cfg.MODEL.WEIGHTS = ''#model_zoo.get_checkpoint_url('COCO-Detection/retinanet_R_101_FPN_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1800
cfg.SOLVER.MAX_ITER = 14400
cfg.SOLVER.STEPS = (1800, 9000)
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.SOLVER.CHECKPOINT_PERIOD = 2500 
cfg.TEST.EVAL_PERIOD = 3600
cfg.OUTPUT_DIR = '/content/drive/My Drive/retinanet_detectron/try03'

Loading config /usr/local/lib/python3.6/dist-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [ ]:
from detectron2.engine import DefaultTrainer
cfg = get_cfg()
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/retinanet_R_101_FPN_3x.yaml'))
cfg.MODEL.WEIGHTS = '/content/drive/My Drive/retinanet_detectron/try02/model_0003599.pth'
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

Loading config /usr/local/lib/python3.6/dist-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [ ]:
import cv2
l = os.listdir('/content/fisheye-day-30062020/images/train')


In [ ]:
import time
from tqdm import tqdm
start = time.time()
for i in tqdm(range(5000)):
  im = cv2.imread('/content/fisheye-day-30062020/images/train/' + l[i])
  outputs = predictor(im)
end = time.time()
print(end-start)

100%|██████████| 5000/5000 [08:31<00:00,  9.78it/s]

511.0207345485687


In [ ]:
511/5000

0.1022

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume = False)
trainer.train()

[08/04 06:01:32 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fc28794efd0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 962, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 50, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


[08/04 06:17:18 d2.data.datasets.coco]: Loading /content/train_labels_cocoformat.json takes 946.19 seconds.
[08/04 06:17:18 d2.data.datasets.coco]: Loaded 14473 images in COCO format from /content/train_labels_cocoformat.json
[08/04 06:17:18 d2.data.build]: Removed 0 images with no usable annotations. 14473 images left.
[08/04 06:17:19 d2.data.common]: Serializing 14473 elements to byte tensors and concatenating them all ...
[08/04 06:17:19 d2.data.common]: Serialized dataset takes 10.99 MiB
[08/04 06:17:19 d2.data.dataset_mapper]: Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[08/04 06:17:19 d2.data.build]: Using training sampler TrainingSampler
[08/04 06:17:19 d2.engine.train_loop]: Starting training from iteration 0
ERROR [08/04 06:17:28 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/detectro

FloatingPointError: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/My Drive/retinanet_detectron/try02'

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.7
cfg.DATASETS.TEST = ('vip_val',)
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator('vip_val', cfg, False, output_dir = '/content/')
val_loader = build_detection_test_loader(cfg, 'vip_val')
print(inference_on_dataset(trainer.model, val_loader, evaluator))

[08/03 17:31:52 d2.data.datasets.coco]: Loaded 5348 images in COCO format from /content/val_labels_cocoformat.json
[08/03 17:31:52 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|  vehicle   | 95693        |
|            |              |
[08/03 17:31:52 d2.data.common]: Serializing 5348 elements to byte tensors and concatenating them all ...
[08/03 17:31:52 d2.data.common]: Serialized dataset takes 6.36 MiB
[08/03 17:31:52 d2.data.dataset_mapper]: Augmentations used in training: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/03 17:31:52 d2.evaluation.evaluator]: Start inference on 5348 images
[08/03 17:31:54 d2.evaluation.evaluator]: Inference done 11/5348. 0.0999 s / img. ETA=0:08:58
[08/03 17:31:59 d2.evaluation.evaluator]: Inference done 61/5348. 0.0997 s / img. ETA=0:08:56
[08/03 17:32:04 d2.evaluation.evaluator]: Inference done 111/5348. 0.0999 s / img. ETA=